# Comparing channels

- R: sequential
- G: reverse complementary
- B: not match at all

In [1]:
import ppscore
import pandas
import os
import numpy
from scipy.stats import spearmanr
from IPython.core.display import display, HTML
from ete3 import Tree

/tmp/ipykernel_50469/499083497.py:6: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [9]:
tree_path = "../../../data/trees"
channels = ("full", "gray_r", "gray_g", "gray_b", "gray_mean")
datasets = ["orthologs_hemoglobin_beta", "orthologs_myoglobin", "orthologs_neuroglobin", "orthologs_cytoglobin"]

In [10]:
def read_and_compare(dataset, channel="full"):
    result_dict = {}
    path = f"{tree_path}/{channel}/{dataset}"
    control_df = pandas.read_csv(f"{path}/Control with Clustal Omega.csv", index_col=0)
    for file in os.listdir(path):
        if file.endswith(".csv"):
            basename = ".".join(file.split(".")[0:-1])
            try:
                df = pandas.read_csv(f"{path}/{file}", index_col=0)
                result_dict[basename] = round(numpy.sqrt(numpy.sum((control_df.values - df.values)**2)), 4)
            except FileNotFoundError:
                pass
    result_df = pandas.DataFrame(result_dict, index=[dataset])
    result_df.rename(columns={"MultiScale Structural Similarity Index Measure": channel}, inplace=True)
    result_df.index.name = "dataset"
    return result_df

## Euclidean Distance from distance matrices

In [11]:
sum_dfs = []
for channel in channels:
    dfs = []
    for dataset in datasets:
        dfs.append(read_and_compare(dataset, channel))
    sum_dfs.append(pandas.concat(dfs))
pandas.concat(sum_dfs, axis=1).T.reset_index().drop_duplicates(["index"]).set_index("index").T

index,full,Global with Needleman-Wunsch,Control with Clustal Omega,Local with Smith–Waterman,gray_r,gray_g,gray_b,gray_mean
dataset,,,,,,,,
orthologs_hemoglobin_beta,1.4402,0.9085,0.0,0.8937,1.1881,1.4400,1.6967,2.0156
orthologs_myoglobin,1.7112,0.9196,0.0,0.9044,1.6690,1.5089,1.9591,2.4938
orthologs_neuroglobin,3.5834,1.8043,0.0,1.1514,1.8423,1.8865,3.8615,4.2052
orthologs_cytoglobin,7.9091,6.3417,0.0,3.1095,5.6928,5.6750,8.1297,8.1834


In [12]:
def read_and_linear_correlate(dataset, channel="full"):
    result_dict = {}
    path = f"{tree_path}/{channel}/{dataset}"
    control_df = pandas.read_csv(f"{path}/Control with Clustal Omega.csv", index_col=0)
    for file in os.listdir(path):
        if file.endswith(".csv"):
            basename = ".".join(file.split(".")[0:-1])
            try:
                df = pandas.read_csv(f"{path}/{file}", index_col=0)
                result_dict[basename] = round(spearmanr(control_df.values.flatten(), df.values.flatten())[0], 4)
            except FileNotFoundError:
                pass
    result_df = pandas.DataFrame(result_dict, index=[dataset])
    result_df.rename(columns={"MultiScale Structural Similarity Index Measure": channel}, inplace=True)
    result_df.index.name = "dataset"
    return result_df

## Linear Correlation from distance matrices

In [13]:
sum_dfs = []
for channel in channels:
    dfs = []
    for dataset in datasets:
        dfs.append(read_and_linear_correlate(dataset, channel))
    sum_dfs.append(pandas.concat(dfs))
pandas.concat(sum_dfs, axis=1).T.reset_index().drop_duplicates(["index"]).set_index("index").T

index,full,Global with Needleman-Wunsch,Control with Clustal Omega,Local with Smith–Waterman,gray_r,gray_g,gray_b,gray_mean
dataset,,,,,,,,
orthologs_hemoglobin_beta,0.9912,0.9999,1.0,0.9998,0.9935,0.9880,0.9925,0.9951
orthologs_myoglobin,0.9783,0.9999,1.0,0.9998,0.9736,0.9799,0.9765,0.9817
orthologs_neuroglobin,0.9891,0.9999,1.0,0.9992,0.9901,0.9886,0.9881,0.9875
orthologs_cytoglobin,0.5479,0.7775,1.0,0.8623,0.4893,0.4877,0.5435,0.5329


In [14]:
def read_and_correlate(dataset, channel="full"):
    result_dict = {}
    path = f"{tree_path}/{channel}/{dataset}"
    control_df = pandas.read_csv(f"{path}/Control with Clustal Omega.csv", index_col=0)
    for file in os.listdir(path):
        if file.endswith(".csv"):
            basename = ".".join(file.split(".")[0:-1])
            try:
                df = pandas.read_csv(f"{path}/{file}", index_col=0)
                result_dict[basename] = round(ppscore.score(
                    pandas.DataFrame({"x": control_df.values.flatten(), "y": df.values.flatten()}), "x", "y")["ppscore"], 4)
            except FileNotFoundError:
                pass
    result_df = pandas.DataFrame(result_dict, index=[dataset])
    result_df.rename(columns={"MultiScale Structural Similarity Index Measure": channel}, inplace=True)
    result_df.index.name = "dataset"
    return result_df

## Predictive Power Score from distance matrices

In [15]:
sum_dfs = []
for channel in channels:
    dfs = []
    for dataset in datasets:
        dfs.append(read_and_correlate(dataset, channel))
    sum_dfs.append(pandas.concat(dfs))
pandas.concat(sum_dfs, axis=1).T.reset_index().drop_duplicates(["index"]).set_index("index").T

index,full,Global with Needleman-Wunsch,Control with Clustal Omega,Local with Smith–Waterman,gray_r,gray_g,gray_b,gray_mean
dataset,,,,,,,,
orthologs_hemoglobin_beta,0.9545,0.9953,0.9968,0.9944,0.9482,0.9509,0.9538,0.9576
orthologs_myoglobin,0.9252,0.9954,0.9968,0.9925,0.9257,0.9142,0.9242,0.9417
orthologs_neuroglobin,0.9625,0.9876,0.9964,0.9897,0.9702,0.9609,0.9602,0.9568
orthologs_cytoglobin,0.7825,0.8524,0.9913,0.8584,0.7471,0.7615,0.7766,0.7916


In [16]:
def read_and_tree_compare(dataset, channel="full", control="Control with Clustal Omega"):
    result_dict = {}
    path = f"{tree_path}/{channel}/{dataset}"
    control_tree = Tree(f"{path}/{control}.nw", format=1)
    for file in os.listdir(path):
        if file.endswith(".nw"):
            basename = ".".join(file.split(".")[0:-1])
            try:
                tree = Tree(f"{path}/{file}", format=1)
                result = control_tree.compare(tree, unrooted=True)
                result_dict[basename] = round(result["norm_rf"], 4)
            except FileNotFoundError:
                pass
    result_df = pandas.DataFrame(result_dict, index=[dataset])
    result_df.rename(columns={"MultiScale Structural Similarity Index Measure": channel}, inplace=True)
    result_df.index.name = "dataset"
    return result_df

## Robinson-foulds distance from generated Trees

In [17]:
sum_dfs = []
for channel in channels:
    dfs = []
    for dataset in datasets:
        dfs.append(read_and_tree_compare(dataset, channel))
    sum_dfs.append(pandas.concat(dfs))
pandas.concat(sum_dfs, axis=1).T.reset_index().drop_duplicates(["index"]).set_index("index").T

index,Global with Needleman-Wunsch,Local with Smith–Waterman,full,Control with Clustal Omega,gray_r,gray_g,gray_b,gray_mean
dataset,,,,,,,,
orthologs_hemoglobin_beta,0.0000,0.0,0.0000,0.0,0.0833,0.0000,0.0833,0.0833
orthologs_myoglobin,0.0000,0.0,0.0000,0.0,0.0000,0.0000,0.0000,0.0000
orthologs_neuroglobin,0.0000,0.0,0.0833,0.0,0.3333,0.4167,0.0833,0.0833
orthologs_cytoglobin,0.6667,0.5,0.5833,0.0,0.4167,0.5833,0.6667,0.6667


In [18]:
sum_dfs = []
for channel in channels:
    dfs = []
    for dataset in datasets:
        dfs.append(read_and_tree_compare(dataset, channel, "Global with Needleman-Wunsch"))
    sum_dfs.append(pandas.concat(dfs))
pandas.concat(sum_dfs, axis=1).T.reset_index().drop_duplicates(["index"]).set_index("index").T

index,Global with Needleman-Wunsch,Local with Smith–Waterman,full,Control with Clustal Omega,gray_r,gray_g,gray_b,gray_mean
dataset,,,,,,,,
orthologs_hemoglobin_beta,0.0,0.00,0.0000,0.0000,0.0833,0.0000,0.0833,0.0833
orthologs_myoglobin,0.0,0.00,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
orthologs_neuroglobin,0.0,0.00,0.0833,0.0000,0.3333,0.4167,0.0833,0.0833
orthologs_cytoglobin,0.0,0.25,0.4167,0.6667,0.5833,0.6667,0.3333,0.3333


In [19]:
sum_dfs = []
for channel in channels:
    dfs = []
    for dataset in datasets:
        dfs.append(read_and_tree_compare(dataset, channel, "Local with Smith–Waterman"))
    sum_dfs.append(pandas.concat(dfs))
pandas.concat(sum_dfs, axis=1).T.reset_index().drop_duplicates(["index"]).set_index("index").T

index,Global with Needleman-Wunsch,Local with Smith–Waterman,full,Control with Clustal Omega,gray_r,gray_g,gray_b,gray_mean
dataset,,,,,,,,
orthologs_hemoglobin_beta,0.00,0.0,0.0000,0.0,0.0833,0.0000,0.0833,0.0833
orthologs_myoglobin,0.00,0.0,0.0000,0.0,0.0000,0.0000,0.0000,0.0000
orthologs_neuroglobin,0.00,0.0,0.0833,0.0,0.3333,0.4167,0.0833,0.0833
orthologs_cytoglobin,0.25,0.0,0.3333,0.5,0.4167,0.5833,0.2500,0.2500


In [20]:
def read_and_tree_compare_branches(dataset, channel="full"):
    result_dict = {}
    path = f"{tree_path}/{channel}/{dataset}"
    control_tree = Tree(f"{path}/Control with Clustal Omega.nw", format=1)
    for file in os.listdir(path):
        if file.endswith(".nw"):
            basename = ".".join(file.split(".")[0:-1])
            try:
                tree = Tree(f"{path}/{file}", format=1)
                result = control_tree.compare(tree, unrooted=True)
                result_dict[basename] = 1.0 - round(result["source_edges_in_ref"], 4)
            except FileNotFoundError:
                pass
    result_df = pandas.DataFrame(result_dict, index=[dataset])
    result_df.rename(columns={"MultiScale Structural Similarity Index Measure": channel}, inplace=True)
    result_df.index.name = "dataset"
    return result_df

## Compatibility branch score from generated Trees

In [21]:
sum_dfs = []
for channel in channels:
    dfs = []
    for dataset in datasets:
        dfs.append(read_and_tree_compare_branches(dataset, channel))
    sum_dfs.append(pandas.concat(dfs))
pandas.concat(sum_dfs, axis=1).T.reset_index().drop_duplicates(["index"]).set_index("index").T

index,Global with Needleman-Wunsch,Local with Smith–Waterman,full,Control with Clustal Omega,gray_r,gray_g,gray_b,gray_mean
dataset,,,,,,,,
orthologs_hemoglobin_beta,0.0000,0.0000,0.0000,0.0,0.0385,0.0000,0.0385,0.0385
orthologs_myoglobin,0.0000,0.0000,0.0000,0.0,0.0000,0.0000,0.0000,0.0000
orthologs_neuroglobin,0.0000,0.0000,0.0385,0.0,0.1538,0.1923,0.0385,0.0385
orthologs_cytoglobin,0.3077,0.2308,0.2692,0.0,0.1923,0.2692,0.3077,0.3077


In [23]:
def read_and_compare_channels(dataset):
    result_dict = {}
    dfs = {}
    for channel in channels:
        result_dict[channel] = {}
        path = f"{tree_path}/{channel}/{dataset}"
        dfs[channel] = pandas.read_csv(f"{path}/MultiScale Structural Similarity Index Measure.csv", index_col=0)
    for channel in channels:
        control_df = dfs[channel]
        for c, df in dfs.items():
            result_dict[channel][c] = round(numpy.sqrt(numpy.sum((control_df.values - df.values)**2)), 4)
    result_df = pandas.DataFrame.from_dict(result_dict, orient='index')
    result_df.index.name = "dataset"
    return result_df.style.background_gradient(axis=None, vmin=result_df.min().min(), vmax=result_df.max().max(), cmap="YlGnBu")

## Features Euclidean distance from distance matrices

In [24]:
for dataset in datasets:
    try:
        correlations = read_and_compare_channels(dataset)
    except FileNotFoundError:
        continue
    else:
        display(HTML(f"<center><h3>{dataset}</h3></center>"))
        display(correlations)        

,full,gray_r,gray_g,gray_b,gray_mean
dataset,,,,,
full,0.000000,0.264000,0.060000,0.261200,0.589200
gray_r,0.264000,0.000000,0.280100,0.521800,0.835600
gray_g,0.060000,0.280100,0.000000,0.271800,0.601500
gray_b,0.261200,0.521800,0.271800,0.000000,0.350400
gray_mean,0.589200,0.835600,0.601500,0.350400,0.000000


,full,gray_r,gray_g,gray_b,gray_mean
dataset,,,,,
full,0.000000,0.072700,0.214800,0.252100,0.796000
gray_r,0.072700,0.000000,0.198300,0.302500,0.841000
gray_g,0.214800,0.198300,0.000000,0.462600,0.998500
gray_b,0.252100,0.302500,0.462600,0.000000,0.557800
gray_mean,0.796000,0.841000,0.998500,0.557800,0.000000


,full,gray_r,gray_g,gray_b,gray_mean
dataset,,,,,
full,0.000000,2.163400,2.170900,0.317300,0.739500
gray_r,2.163400,0.000000,0.103400,2.444300,2.838800
gray_g,2.170900,0.103400,0.000000,2.449000,2.842500
gray_b,0.317300,2.444300,2.449000,0.000000,0.440700
gray_mean,0.739500,2.838800,2.842500,0.440700,0.000000


,full,gray_r,gray_g,gray_b,gray_mean
dataset,,,,,
full,0.000000,2.907000,2.968600,0.268700,0.642600
gray_r,2.907000,0.000000,0.327900,3.148200,3.436400
gray_g,2.968600,0.327900,0.000000,3.216200,3.500400
gray_b,0.268700,3.148200,3.216200,0.000000,0.502100
gray_mean,0.642600,3.436400,3.500400,0.502100,0.000000
